In [7]:
import pandas as pd
pd.set_option('display.max_columns', 100)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, FunctionTransformer, LabelEncoder, RobustScaler
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold, KFold

import lightgbm

from pycaret.classification import *
import shap

from sklearn.metrics import f1_score, accuracy_score, classification_report




In [2]:
from function.data_engineering import *

In [3]:
data = pd.read_csv('penyisihan-satria-data-itb-2023/train.csv')
test = pd.read_csv('penyisihan-satria-data-itb-2023/test.csv')
data.index = data.id
test.index = test.id
data = data.iloc[:,1:]
test = test.iloc[:,1:]

In [4]:
or_cols = [c for c in data.columns if c != 'target']
int_cols = [c for c in data.columns if data[c].dtype=='int64' and c != 'target']
float_cols = [c for c in data.columns if data[c].dtype=='float64' and c != 'target']

In [6]:
for c in int_cols:
    print(c,data[c].nunique())

hs 34
hst 16
hf 19
hc 17
hy 7
hr 3
bb_ou 17
bb_ah 11


In [134]:
# for c in float_cols:
#     print(c,data[c].value_counts())

In [135]:
# tmp_train = data.copy()
# for i in int_cols:
#     for c in int_cols:
#         tmp_train[f'{c}_binding_encoded'] = pd.cut(tmp_train[c], bins=5, labels=[1,2,3,4,5])

In [136]:
for col in int_cols:
  tmp_train = freq_encoding(data, col)
tmp_train

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,...,pscd,target,hs_encode,hst_encode,hf_encode,hc_encode,hy_encode,hr_encode,bb_ou_encode,bb_ah_encode
id,,,,,,,,,,,,,,,,,,,,,
1,23,6,7,10,2,0,1.95,3.50,1.91,3.50,...,3.39,0,0.016667,0.108333,0.071667,0.075000,0.258333,0.955,0.070000,0.138333
2,21,5,5,8,1,0,1.28,6.50,1.26,6.00,...,5.25,0,0.025000,0.136667,0.045000,0.091667,0.301667,0.955,0.140000,0.218333
3,17,6,8,7,1,0,1.28,6.50,1.28,5.75,...,6.11,0,0.040000,0.108333,0.098333,0.108333,0.301667,0.955,0.146667,0.085000
4,26,11,12,6,1,0,1.90,3.79,1.83,3.75,...,3.56,0,0.005000,0.025000,0.111667,0.110000,0.301667,0.955,0.053333,0.200000
5,20,6,11,3,3,0,2.50,3.20,2.50,3.10,...,3.10,0,0.043333,0.108333,0.128333,0.113333,0.131667,0.955,0.161667,0.218333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,16,5,7,5,0,0,1.55,4.20,1.57,4.00,...,4.35,1,0.070000,0.136667,0.071667,0.143333,0.248333,0.955,0.146667,0.183333
597,21,12,5,7,1,0,1.16,8.50,1.16,7.50,...,9.75,1,0.025000,0.015000,0.045000,0.108333,0.301667,0.955,0.048333,0.183333
598,15,8,12,5,3,0,1.90,3.50,1.90,3.40,...,3.51,1,0.076667,0.056667,0.111667,0.143333,0.131667,0.955,0.053333,0.200000


In [137]:
# tmp_train[or_cols] = (tmp_train[or_cols] - tmp_train[or_cols].min())/tmp_train[or_cols].max()-tmp_train[or_cols].min()
scaler = StandardScaler()
scaler.fit(tmp_train[or_cols])
tmp_train[or_cols] = scaler.fit_transform(tmp_train[or_cols])
test[or_cols] = scaler.transform(test[or_cols])

In [138]:
exp_name = setup(data = tmp_train, target = 'target',fold=5,session_id = 42)
exp_name

,Description,Value
0,Session id,42
1,Target,target
2,Target type,Binary
3,Original data shape,"(600, 44)"
4,Transformed data shape,"(600, 44)"
5,Transformed train set shape,"(420, 44)"
6,Transformed test set shape,"(180, 44)"
7,Numeric features,43
8,Preprocess,True
9,Imputation type,simple


In [139]:
lr = create_model('ridge')
tuned_lr = tune_model(lr)


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:48:43
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7381,0.0000,0.6667,0.7778,0.7179,0.4762,0.4811
1,0.6786,0.0000,0.7143,0.6667,0.6897,0.3571,0.3581
2,0.7143,0.0000,0.6429,0.7500,0.6923,0.4286,0.4330
3,0.6310,0.0000,0.5714,0.6486,0.6076,0.2619,0.2638
4,0.7381,0.0000,0.6905,0.7632,0.7250,0.4762,0.4784
Mean,0.7000,0.0000,0.6571,0.7213,0.6865,0.4000,0.4029
Std,0.0408,0.0000,0.0490,0.0530,0.0418,0.0816,0.0826


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:48:49
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Ridge Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7381,0.0000,0.6667,0.7778,0.7179,0.4762,0.4811
1,0.7143,0.0000,0.7381,0.7045,0.7209,0.4286,0.4291
2,0.7024,0.0000,0.6190,0.7429,0.6753,0.4048,0.4105
3,0.6310,0.0000,0.5714,0.6486,0.6076,0.2619,0.2638
4,0.7143,0.0000,0.6905,0.7250,0.7073,0.4286,0.4291
Mean,0.7000,0.0000,0.6571,0.7198,0.6858,0.4000,0.4027
Std,0.0364,0.0000,0.0575,0.0429,0.0423,0.0728,0.0733


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [140]:
#evaluate_model(tuned_lr)

In [141]:
top = compare_models(n_select = 3)
blender = blend_models(top)
tuned_blender = tune_model(blender)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:49:21
Status,. . . . . . . . . . . . . . . . . .,Loading Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.0000,0.6667,0.8000,0.7273,0.5000,0.5071
1,0.6905,0.0000,0.7381,0.6739,0.7045,0.3810,0.3827
2,0.6786,0.0000,0.5952,0.7143,0.6494,0.3571,0.3622
3,0.6429,0.0000,0.5714,0.6667,0.6154,0.2857,0.2887
4,0.7381,0.0000,0.6905,0.7632,0.7250,0.4762,0.4784
Mean,0.7000,0.0000,0.6524,0.7236,0.6843,0.4000,0.4038
Std,0.0394,0.0000,0.0614,0.0514,0.0445,0.0788,0.0796


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:50:18
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.0000,0.6667,0.8000,0.7273,0.5000,0.5071
1,0.6905,0.0000,0.7381,0.6739,0.7045,0.3810,0.3827
2,0.6786,0.0000,0.5952,0.7143,0.6494,0.3571,0.3622
3,0.6429,0.0000,0.5714,0.6667,0.6154,0.2857,0.2887
4,0.7381,0.0000,0.6905,0.7632,0.7250,0.4762,0.4784
Mean,0.7000,0.0000,0.6524,0.7236,0.6843,0.4000,0.4038
Std,0.0394,0.0000,0.0614,0.0514,0.0445,0.0788,0.0796


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:50:22
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7500,0.0000,0.6667,0.8000,0.7273,0.5000,0.5071
1,0.6905,0.0000,0.7381,0.6739,0.7045,0.3810,0.3827
2,0.6786,0.0000,0.5952,0.7143,0.6494,0.3571,0.3622
3,0.6429,0.0000,0.5714,0.6667,0.6154,0.2857,0.2887
4,0.7381,0.0000,0.6905,0.7632,0.7250,0.4762,0.4784
Mean,0.7000,0.0000,0.6524,0.7236,0.6843,0.4000,0.4038
Std,0.0394,0.0000,0.0614,0.0514,0.0445,0.0788,0.0796


Fitting 5 folds for each of 10 candidates, totalling 50 fits


In [142]:
X = [c for c in tmp_train.columns if c != 'target']
train = tmp_train[X].copy()
predict_model(tuned_lr,train)

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,...,pscd,hs_encode,hst_encode,hf_encode,hc_encode,hy_encode,hr_encode,bb_ou_encode,bb_ah_encode,prediction_label
id,,,,,,,,,,,,,,,,,,,,,
1,1.524832,0.434034,-1.012809,1.429838,0.430124,-0.213399,-0.418164,-0.540222,-0.430541,-0.505686,...,-0.585915,0.016667,0.108333,0.071667,0.075000,0.258333,0.955,0.070000,0.138333,0
2,1.181208,0.067245,-1.631319,0.749503,-0.386567,-0.213399,-0.673929,1.071365,-0.704861,1.005207,...,0.370583,0.025000,0.136667,0.045000,0.091667,0.301667,0.955,0.140000,0.218333,1
3,0.493960,0.434034,-0.703555,0.409335,-0.386567,-0.213399,-0.673929,1.071365,-0.696420,0.854118,...,0.812835,0.040000,0.108333,0.098333,0.108333,0.301667,0.955,0.146667,0.085000,1
4,2.040268,2.267979,0.533464,0.069167,-0.386567,-0.213399,-0.437251,-0.384435,-0.464304,-0.354596,...,-0.498493,0.005000,0.025000,0.111667,0.110000,0.301667,0.955,0.053333,0.200000,1
5,1.009396,0.434034,0.224210,-0.951336,1.246814,-0.213399,-0.208207,-0.701381,-0.181543,-0.747428,...,-0.735047,0.043333,0.108333,0.128333,0.113333,0.131667,0.955,0.161667,0.218333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
596,0.322148,0.067245,-1.012809,-0.271000,-1.203257,-0.213399,-0.570860,-0.164185,-0.574031,-0.203507,...,-0.092239,0.070000,0.136667,0.071667,0.143333,0.248333,0.955,0.146667,0.183333,1
597,1.181208,2.634768,-1.631319,0.409335,-0.386567,-0.213399,-0.719738,2.145757,-0.747064,1.911742,...,2.684692,0.025000,0.015000,0.045000,0.108333,0.301667,0.955,0.048333,0.183333,1
598,0.150336,1.167612,0.533464,-0.271000,1.246814,-0.213399,-0.437251,-0.540222,-0.434761,-0.566121,...,-0.524206,0.076667,0.056667,0.111667,0.143333,0.131667,0.955,0.053333,0.200000,1


In [143]:
save_model(tuned_lr,model_name='lr-tuned')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=FastMemory(location=/var/folders/yt/n9_2kr5s6nx_5b2y_kx3mnnm0000gn/T/joblib),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['hs', 'hst', 'hf', 'hc', 'hy',
                                              'hr', 'b365h', 'b365d', 'bwh',
                                              'bwd', 'iwh', 'iwd', 'psh', 'psd',
                                              'whh', 'whd', 'vch', 'vcd',
                                              'bb_mx_h', 'bb_av_h', 'bb_mx_d',
                                              'bb_av_d', 'bb_ou', 'bb_mx_2_5',
                                              'bb_av_2_5', 'bb_mx_2_...
                  TransformerWrapper(exclude=None, include=[],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                       

In [144]:
test

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,...,bb_mx_2_5_1,bb_av_2_5_1,bb_ah,bb_a_hh,bb_mx_ahh,bb_av_ahh,bb_mx_aha,bb_av_aha,psch,pscd
id,,,,,,,,,,,,,,,,,,,,,
1,-0.193289,0.434034,0.224210,0.409335,-0.386567,-0.213399,-0.589947,-0.094349,-0.603574,-0.082636,...,-0.503706,-0.478461,0.484512,-0.610598,-0.524994,-0.511267,0.308592,0.361790,-0.504804,-0.503636
2,-1.224161,-0.299544,-1.012809,-0.611168,-0.386567,-0.213399,0.746142,-0.330716,0.768025,-0.354596,...,-0.056284,-0.044876,-0.561197,1.118674,-0.320384,-0.183474,0.092542,0.133050,0.745418,-0.426499
3,0.322148,0.434034,0.224210,0.409335,-1.203257,-0.213399,-0.418164,-0.593942,-0.413660,-0.656775,...,-0.970581,-0.977084,-0.038343,0.130519,-1.599193,-1.549281,1.928973,1.962967,-0.370988,-0.709334
4,-0.021477,-0.666333,-1.631319,1.429838,-1.203257,-0.213399,-0.547955,-0.164185,-0.561371,-0.143071,...,0.254966,0.301992,1.007366,-0.610598,0.395749,0.417482,-0.501598,-0.553169,-0.581270,-0.056241
5,-0.193289,-0.666333,1.461229,-1.291504,0.430123,-0.213399,0.517098,-0.325344,0.620314,-0.354596,...,-0.231362,-0.174952,-1.084052,1.118674,-0.883060,-0.893694,0.740694,0.876454,0.374557,-0.508778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,-0.880537,-1.033122,-1.012809,0.409335,0.430123,-0.213399,-0.284555,-0.701381,-0.287051,-0.747428,...,-1.067847,-1.063801,-0.561197,-0.116520,1.776862,1.892554,-1.527839,-1.582497,-0.325109,-0.637340
157,-0.880537,-1.033122,0.224210,0.069167,0.430123,-0.213399,-0.612851,0.319291,-0.624675,0.249761,...,1.422153,1.429313,2.053075,-0.610598,-1.036517,-1.112223,1.280821,1.105193,-0.615680,0.180312
158,1.181208,0.800823,2.079738,0.749503,-0.386567,-0.213399,-0.360903,-0.540222,-0.350355,-0.566121,...,-0.503706,-0.500140,-0.561197,0.130519,-0.576146,-0.729797,0.848719,0.762084,-0.428338,-0.477923


In [145]:
for col in int_cols:
  tmp_test = freq_encoding(test, col)
tmp_test
# tmp_test[or_cols] = (tmp_test[or_cols] - tmp_test[or_cols].min())/tmp_test[or_cols].max()-tmp_test[or_cols].min()

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,...,psch,pscd,hs_encode,hst_encode,hf_encode,hc_encode,hy_encode,hr_encode,bb_ou_encode,bb_ah_encode
id,,,,,,,,,,,,,,,,,,,,,
1,-0.193289,0.434034,0.224210,0.409335,-0.386567,-0.213399,-0.589947,-0.094349,-0.603574,-0.082636,...,-0.504804,-0.503636,0.06875,0.07500,0.10625,0.08750,0.33125,0.95625,0.31875,0.16875
2,-1.224161,-0.299544,-1.012809,-0.611168,-0.386567,-0.213399,0.746142,-0.330716,0.768025,-0.354596,...,0.745418,-0.426499,0.06875,0.13750,0.08750,0.11875,0.33125,0.95625,0.01250,0.21875
3,0.322148,0.434034,0.224210,0.409335,-1.203257,-0.213399,-0.418164,-0.593942,-0.413660,-0.656775,...,-0.370988,-0.709334,0.05000,0.07500,0.10625,0.08750,0.18750,0.95625,0.10000,0.21250
4,-0.021477,-0.666333,-1.631319,1.429838,-1.203257,-0.213399,-0.547955,-0.164185,-0.561371,-0.143071,...,-0.581270,-0.056241,0.08750,0.15625,0.05625,0.05000,0.18750,0.95625,0.31875,0.11875
5,-0.193289,-0.666333,1.461229,-1.291504,0.430123,-0.213399,0.517098,-0.325344,0.620314,-0.354596,...,0.374557,-0.508778,0.06875,0.15625,0.05000,0.10625,0.30000,0.95625,0.10625,0.11875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,-0.880537,-1.033122,-1.012809,0.409335,0.430123,-0.213399,-0.284555,-0.701381,-0.287051,-0.747428,...,-0.325109,-0.637340,0.06250,0.15000,0.08750,0.08750,0.30000,0.95625,0.31875,0.21875
157,-0.880537,-1.033122,0.224210,0.069167,0.430123,-0.213399,-0.612851,0.319291,-0.624675,0.249761,...,-0.615680,0.180312,0.06250,0.15000,0.10625,0.12500,0.30000,0.95625,0.10625,0.00625
158,1.181208,0.800823,2.079738,0.749503,-0.386567,-0.213399,-0.360903,-0.540222,-0.350355,-0.566121,...,-0.428338,-0.477923,0.02500,0.08125,0.01875,0.10000,0.33125,0.95625,0.18750,0.21875


In [146]:
df = predict_model(tuned_lr,tmp_test)
df

,hs,hst,hf,hc,hy,hr,b365h,b365d,bwh,bwd,...,pscd,hs_encode,hst_encode,hf_encode,hc_encode,hy_encode,hr_encode,bb_ou_encode,bb_ah_encode,prediction_label
id,,,,,,,,,,,,,,,,,,,,,
1,-0.193289,0.434034,0.224210,0.409335,-0.386567,-0.213399,-0.589947,-0.094349,-0.603574,-0.082636,...,-0.503636,0.06875,0.07500,0.10625,0.08750,0.33125,0.95625,0.31875,0.16875,1
2,-1.224161,-0.299544,-1.012809,-0.611168,-0.386567,-0.213399,0.746142,-0.330716,0.768025,-0.354596,...,-0.426499,0.06875,0.13750,0.08750,0.11875,0.33125,0.95625,0.01250,0.21875,0
3,0.322148,0.434034,0.224210,0.409335,-1.203257,-0.213399,-0.418164,-0.593942,-0.413660,-0.656775,...,-0.709334,0.05000,0.07500,0.10625,0.08750,0.18750,0.95625,0.10000,0.21250,1
4,-0.021477,-0.666333,-1.631319,1.429838,-1.203257,-0.213399,-0.547955,-0.164185,-0.561371,-0.143071,...,-0.056241,0.08750,0.15625,0.05625,0.05000,0.18750,0.95625,0.31875,0.11875,1
5,-0.193289,-0.666333,1.461229,-1.291504,0.430124,-0.213399,0.517098,-0.325344,0.620314,-0.354596,...,-0.508778,0.06875,0.15625,0.05000,0.10625,0.30000,0.95625,0.10625,0.11875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156,-0.880537,-1.033122,-1.012809,0.409335,0.430124,-0.213399,-0.284555,-0.701381,-0.287051,-0.747428,...,-0.637340,0.06250,0.15000,0.08750,0.08750,0.30000,0.95625,0.31875,0.21875,0
157,-0.880537,-1.033122,0.224210,0.069167,0.430124,-0.213399,-0.612851,0.319291,-0.624675,0.249761,...,0.180312,0.06250,0.15000,0.10625,0.12500,0.30000,0.95625,0.10625,0.00625,1
158,1.181208,0.800823,2.079738,0.749503,-0.386567,-0.213399,-0.360903,-0.540222,-0.350355,-0.566121,...,-0.477923,0.02500,0.08125,0.01875,0.10000,0.33125,0.95625,0.18750,0.21875,0


In [147]:
df.prediction_label.value_counts()

0    85
1    75
Name: prediction_label, dtype: int64

In [148]:
submit = df['prediction_label'].reset_index()

In [149]:
submit.to_csv('submit.csv',index=False)